In [1]:
from cdag import Graph, load_data, read_table, parse, tokenize
from personal import database_connection
import re, pandas as pd
from collections import defaultdict

In [11]:
# graph = Graph()
# load_data(graph)

In [12]:
# primary_groups: CHIP (CS only), PS (irrelevant), ZKH (tree root), plus some others
# root = graph.find_one_node('Group', searchcode='ZKH')

In [5]:
# read HiX tables
# method 1: from CSV files
# method 2: from database
# connection = database_connection('hixacceptatie')
# iv = pd.read_sql(sql=f"select * from [hix_acc].[dbo].[config_instvars];", con=connection
EMPTY = '-'
iv = pd.read_csv('config_instvars.csv')
iv['VALUE'] = iv['VALUE'].fillna(EMPTY)
iv['NAAM'] = iv['NAAM'].fillna(EMPTY)
iv['SPECCODE'] = iv['SPECCODE'].fillna(EMPTY)
ivcl = iv.loc[(iv['INSTTYPE']=='L') | (iv['INSTTYPE']=='C')]

In [16]:
# read HiX tables (continued)
wc = pd.read_csv('config_workcontext.csv')
wc.set_index('Id', inplace=True)
wc['SettingSubId'] = wc['SettingSubId'].fillna(EMPTY)
ws = pd.read_csv('config_wcsegments.csv')
ws.set_index('ConfiguredWorkContextId', inplace=True)
wsa = ws.loc[ws['Disabled'] == False]
wcs = wc.join(wsa, how='left')
wcs.sort_values(by=['SettingId', 'SettingSubId'], inplace=True)
# gr = pd.read_csv('ziscon_groepen.csv')
# us = pd.read_csv('ziscon_user.csv',
#                  dtype={'FUNKTIE':'string', 'BEHEERDER':'boolean', 'WACHTWOORD':'string',
#                         'DISABLED': 'boolean', 'UZINR': 'string', 'ROLE': 'string', 'WRONGCHECKS': 'string'})

In [20]:
#x=
ivcl.loc[(ivcl['OWNER'] == '@GBEH')]# & iv['NAAM'].str.contains('OND_')]#.to_dict('records')
#parse(x[0]['VALUE'])

In [19]:
wcs.loc[(wcs['OwnerId']=='BEH') & wcs['SegmentClassId'].str.contains('DOSSIER')]